# Learning Embeddings
Experimenting with autoencoder for molecular graph embeddings

In [1]:
#imports
import pandas as pd
from tdc.single_pred import ADME

C:\Users\ninja\AppData\Local\Temp\ipykernel_19236\1866585249.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [14]:
# load data
from tdc.single_pred import ADME
data = ADME(name = 'Solubility_AqSolDB')
data = data.get_split()
train = data['train']
val = data['valid']
test = data['test']
train

Found local copy...
Loading...
Done!


,Drug_ID,Drug,Y
0,Benzo[cd]indol-2(1H)-one,O=C1Nc2cccc3cccc1c23,-3.254767
1,4-chlorobenzaldehyde,O=Cc1ccc(Cl)cc1,-2.177078
2,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,c1cc(N(CC2CO2)CC2CO2)ccc1Cc1ccc(N(CC2CO2)CC2CO...,-4.662065
3,vinyltoluene,C=Cc1cccc(C)c1,-3.123150
4,3-(3-ethylcyclopentyl)propanoic acid,CCC1CCC(CCC(=O)O)C1,-3.286116
...,...,...,...
6983,sarafloxacin,O=C(O)c1cn(-c2ccc(F)cc2)c2cc(N3CCNCC3)c(F)cc2c1=O,-3.130000
6984,sparfloxacin,C[C@H]1CN(c2c(F)c(N)c3c(=O)c(C(=O)O)cn(C4CC4)c...,-3.370000
6985,sulindac_form_II,CC1=C(CC(=O)O)c2cc(F)ccc2/C1=C/c1ccc(S(C)=O)cc1,-4.500000
6986,tetracaine,CCCCNc1ccc(C(=O)OCCN(C)C)cc1,-3.010000


### Data Conversion

Tutorial code from Marcus Deblander, Oxford Protein Informatics Group: https://www.blopig.com/blog/2022/02/how-to-turn-a-smiles-string-into-a-molecular-graph-for-pytorch-geometric/

In [3]:
# import packages

# general tools
import numpy as np

# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

# Pytorch and Pytorch Geometric
import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader

In [4]:
def one_hot_encoding(x, permitted_list):
    """
    Maps input elements x which are not in the permitted list to the last element
    of the permitted list.
    """

    if x not in permitted_list:
        x = permitted_list[-1]

    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]

    return binary_encoding

In [5]:
def get_atom_features(atom, 
                      use_chirality = True, 
                      hydrogens_implicit = True):
    """
    Takes an RDKit atom object as input and gives a 1d-numpy array of atom features as output.
    """

    # define list of permitted atoms
    
    permitted_list_of_atoms =  ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca','Fe','As','Al','I', 'B','V','K','Tl','Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn', 'Li','Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr','Pt','Hg','Pb','Unknown']
    
    if hydrogens_implicit == False:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms
    
    # compute atom features
    
    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list_of_atoms)
    
    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])
    
    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])
    
    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])
    
    is_in_a_ring_enc = [int(atom.IsInRing())]
    
    is_aromatic_enc = [int(atom.GetIsAromatic())]
    
    atomic_mass_scaled = [float((atom.GetMass() - 10.812)/116.092)]
    
    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)]
    
    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)]

    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + covalent_radius_scaled
                                    
    if use_chirality == True:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc
    
    if hydrogens_implicit == True:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc

    return np.array(atom_feature_vector)

In [6]:
def get_bond_features(bond, 
                      use_stereochemistry = True):
    """
    Takes an RDKit bond object as input and gives a 1d-numpy array of bond features as output.
    """

    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]

    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)
    
    bond_is_conj_enc = [int(bond.GetIsConjugated())]
    
    bond_is_in_ring_enc = [int(bond.IsInRing())]
    
    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc
    
    if use_stereochemistry == True:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc

    return np.array(bond_feature_vector)

In [7]:
def create_pytorch_geometric_graph_data_list_from_smiles_and_labels(x_smiles, y):
    """
    Inputs:
    
    x_smiles = [smiles_1, smiles_2, ....] ... a list of SMILES strings
    y = [y_1, y_2, ...] ... a list of numerial labels for the SMILES strings (such as associated pKi values)
    
    Outputs:
    
    data_list = [G_1, G_2, ...] ... a list of torch_geometric.data.Data objects which represent labeled molecular graphs that can readily be used for machine learning
    
    """
    
    data_list = []
    
    for (smiles, y_val) in zip(x_smiles, y):
        
        # convert SMILES to RDKit mol object
        mol = Chem.MolFromSmiles(smiles)

        # get feature dimensions
        n_nodes = mol.GetNumAtoms()
        n_edges = 2*mol.GetNumBonds()
        unrelated_smiles = "O=O"
        unrelated_mol = Chem.MolFromSmiles(unrelated_smiles)
        n_node_features = len(get_atom_features(unrelated_mol.GetAtomWithIdx(0)))
        n_edge_features = len(get_bond_features(unrelated_mol.GetBondBetweenAtoms(0,1)))

        # construct node feature matrix X of shape (n_nodes, n_node_features)
        X = np.zeros((n_nodes, n_node_features))

        for atom in mol.GetAtoms():
            X[atom.GetIdx(), :] = get_atom_features(atom)
            
        X = torch.tensor(X, dtype = torch.float)
        
        # construct edge index array E of shape (2, n_edges)
        (rows, cols) = np.nonzero(GetAdjacencyMatrix(mol))
        torch_rows = torch.from_numpy(rows.astype(np.int64)).to(torch.long)
        torch_cols = torch.from_numpy(cols.astype(np.int64)).to(torch.long)
        E = torch.stack([torch_rows, torch_cols], dim = 0)
        
        # construct edge feature array EF of shape (n_edges, n_edge_features)
        EF = np.zeros((n_edges, n_edge_features))
        
        for (k, (i,j)) in enumerate(zip(rows, cols)):
            
            EF[k] = get_bond_features(mol.GetBondBetweenAtoms(int(i),int(j)))
        
        EF = torch.tensor(EF, dtype = torch.float)
        
        # construct label tensor
        y_tensor = torch.tensor(np.array([y_val]), dtype = torch.float)
        
        # construct Pytorch Geometric data object and append to data list
        data_list.append(Data(x = X, edge_index = E, edge_attr = EF, y = y_tensor))

    return data_list

In [15]:
# sanity check 
create_pytorch_geometric_graph_data_list_from_smiles_and_labels(train['Drug'], train['Y'])

[09:11:41] WARNING: not removing hydrogen atom without neighbors
[09:11:41] WARNING: not removing hydrogen atom without neighbors
[09:11:41] WARNING: not removing hydrogen atom without neighbors
[09:11:41] WARNING: not removing hydrogen atom without neighbors
[09:11:42] WARNING: not removing hydrogen atom without neighbors
[09:11:42] WARNING: not removing hydrogen atom without neighbors
[09:11:43] WARNING: not removing hydrogen atom without neighbors
[09:11:43] WARNING: not removing hydrogen atom without neighbors
[09:11:43] WARNING: not removing hydrogen atom without neighbors
[09:11:43] WARNING: not removing hydrogen atom without neighbors
[09:11:43] WARNING: not removing hydrogen atom without neighbors
[09:11:43] WARNING: not removing hydrogen atom without neighbors
[09:11:43] WARNING: not removing hydrogen atom without neighbors
[09:11:43] WARNING: not removing hydrogen atom without neighbors
[09:11:43] WARNING: not removing hydrogen atom without neighbors
[09:11:43] WARNING: not r

[Data(x=[13, 79], edge_index=[2, 30], edge_attr=[30, 10], y=[1]),
 Data(x=[9, 79], edge_index=[2, 18], edge_attr=[18, 10], y=[1]),
 Data(x=[31, 79], edge_index=[2, 72], edge_attr=[72, 10], y=[1]),
 Data(x=[9, 79], edge_index=[2, 18], edge_attr=[18, 10], y=[1]),
 Data(x=[12, 79], edge_index=[2, 24], edge_attr=[24, 10], y=[1]),
 Data(x=[27, 79], edge_index=[2, 60], edge_attr=[60, 10], y=[1]),
 Data(x=[25, 79], edge_index=[2, 52], edge_attr=[52, 10], y=[1]),
 Data(x=[14, 79], edge_index=[2, 28], edge_attr=[28, 10], y=[1]),
 Data(x=[15, 79], edge_index=[2, 32], edge_attr=[32, 10], y=[1]),
 Data(x=[11, 79], edge_index=[2, 20], edge_attr=[20, 10], y=[1]),
 Data(x=[14, 79], edge_index=[2, 28], edge_attr=[28, 10], y=[1]),
 Data(x=[11, 79], edge_index=[2, 22], edge_attr=[22, 10], y=[1]),
 Data(x=[10, 79], edge_index=[2, 20], edge_attr=[20, 10], y=[1]),
 Data(x=[10, 79], edge_index=[2, 12], edge_attr=[12, 10], y=[1]),
 Data(x=[13, 79], edge_index=[2, 26], edge_attr=[26, 10], y=[1]),
 Data(x=[31,

In [16]:
try_me = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(train['Drug'], train['Y'])
try_me[0].x

[09:26:16] WARNING: not removing hydrogen atom without neighbors
[09:26:16] WARNING: not removing hydrogen atom without neighbors
[09:26:16] WARNING: not removing hydrogen atom without neighbors
[09:26:16] WARNING: not removing hydrogen atom without neighbors
[09:26:16] WARNING: not removing hydrogen atom without neighbors
[09:26:17] WARNING: not removing hydrogen atom without neighbors
[09:26:17] WARNING: not removing hydrogen atom without neighbors
[09:26:17] WARNING: not removing hydrogen atom without neighbors
[09:26:18] WARNING: not removing hydrogen atom without neighbors
[09:26:18] WARNING: not removing hydrogen atom without neighbors
[09:26:18] WARNING: not removing hydrogen atom without neighbors
[09:26:18] WARNING: not removing hydrogen atom without neighbors
[09:26:18] WARNING: not removing hydrogen atom without neighbors
[09:26:18] WARNING: not removing hydrogen atom without neighbors
[09:26:18] WARNING: not removing hydrogen atom without neighbors
[09:26:18] WARNING: not r

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

### Constructing Initial Variational Auto Encoder